In [9]:
import simul_generate
import os
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import mode
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [10]:
ts = datetime.now().strftime('%Y%m%d_%H%M%S')

In [11]:
#eng 
def plot_simulation_with_historical_and_trajectories(
    data_frame, simulation_dict, start_date, plot_trajectories=False, 
    output_pdf='macro_factors.pdf', 
    output_excel='simulation_statistics.xlsx',
    trim_simulation=True):
    
    # Create variable mapping dictionary
    mapping = {
        'cost': 'cost_qoq',
        'cir': 'rate',
        'hpi': 'hpi_pr_qoq',
        'mortgage_rate': 'MR',
        'hpi_second': 'hpi_yoy'
    }
    
    # Optionally trim simulation data
    if trim_simulation:
        trimmed_simulation_dict = {key: data[:, :-15] for key, data in simulation_dict.items()}
    else:
        trimmed_simulation_dict = simulation_dict
    
    # Create dates for simulations
    any_key = next(iter(trimmed_simulation_dict))
    end_simulation_months = trimmed_simulation_dict[any_key].shape[1]
    simulation_dates = pd.date_range(start=start_date, periods=end_simulation_months, freq='M')
    simulation_dates = pd.to_datetime(simulation_dates)
    statistics_dict = {}

    # Dictionary for statistics descriptions
    stats_description = {
        'Median': 'Median of simulation values at each time point',
        'Mean': 'Mean of simulation values at each time point',
        'Mode': 'Mode of simulation values at each time point',
        'StdDev': 'Standard deviation of simulation values at each time point',
        'Q05': '5th percentile of simulation values at each time point',
        'Q95': '95th percentile of simulation values at each time point',
        'MeanAboveQ95': 'Mean of simulations above 95th percentile at each time point',
        'MeanBelowQ05': 'Mean of simulations below 5th percentile at each time point',
        'Mean+StdDev': 'Sum of mean and standard deviation at each time point',
        'Mean-StdDev': 'Difference between mean and standard deviation at each time point',
        'Best_Scenario_Mean': 'Mean of best scenarios',
        'Worst_Scenario_Mean': 'Mean of worst scenarios',
        'VaR_Scenario_Mean': 'Mean of VaR scenarios at 95% level',
        'ES_Scenario_Mean': 'Mean of ES scenarios above VaR 95%',
        'Median_Scenario_Mean': 'Mean of scenarios close to median'
    }

    with PdfPages(output_pdf) as pdf: 
        for simulation_name, simulation_data in trimmed_simulation_dict.items():
            fig, axs = plt.subplots(2 if plot_trajectories else 1, 1, 
                                    figsize=(12, 12 if plot_trajectories else 6), sharex=True)
            if not isinstance(axs, np.ndarray):
                axs = [axs]

            historical_data_plotted = False

            # Check for corresponding column in historical data
            if simulation_name in data_frame.columns:
                historical_column = simulation_name
            elif simulation_name in mapping and mapping[simulation_name] in data_frame.columns:
                historical_column = mapping[simulation_name]
            else:
                historical_column = None  # No match

            # If there's a corresponding column, plot historical data
            if historical_column is not None:
                axs[0].plot(data_frame.Period_Date, data_frame[historical_column], 
                            label='Historical Data', color='blue')
                historical_data_plotted = True

            # Calculate statistics
            simulation_mean = np.mean(simulation_data, axis=0)
            simulation_median = np.median(simulation_data, axis=0)
            simulation_mode = mode(simulation_data, axis=0)[0].squeeze()
            simulation_q05 = np.percentile(simulation_data, 5, axis=0)
            simulation_q95 = np.percentile(simulation_data, 95, axis=0)
            simulation_std = np.std(simulation_data, axis=0)

            # Metric for scenario identification (using mean of first 25 periods)
            metric = np.mean(simulation_data[:, :25], axis=1)

            # Calculate metric quantiles
            metric_q95 = np.percentile(metric, 95)
            metric_q05 = np.percentile(metric, 5)
            
            # Calculate mean values for metric values above 95% and below 5%
            mean_above_q95 = np.mean(simulation_data[metric > metric_q95], axis=0)
            mean_below_q05 = np.mean(simulation_data[metric < metric_q05], axis=0)

            # Define positive metric
            positive_metric = simulation_name in ['gdp_yoy']

            # Calculate best and worst scenarios from first realization
            if positive_metric:
                best_indices = np.argsort(-metric)[:10]  # Indices of 10 largest values
                worst_indices = np.argsort(metric)[:10]  # Indices of 10 smallest values
            else:
                best_indices = np.argsort(metric)[:10]   # Indices of 10 smallest values
                worst_indices = np.argsort(-metric)[:10] # Indices of 10 largest values

            best_scenario_mean = np.mean(simulation_data[best_indices], axis=0)
            worst_scenario_mean = np.mean(simulation_data[worst_indices], axis=0)

            # Calculate VaR and ES from second realization
            var_level = 95
            VaR = np.percentile(metric, var_level)
            ES_indices = np.where(metric > VaR)[0]
            VaR_indices = np.argsort(np.abs(metric - VaR))[:5]  # 5 scenarios closest to VaR
            ES = metric[metric > VaR].mean() if len(ES_indices) > 0 else np.nan

            VaR_scenario_mean = np.mean(simulation_data[VaR_indices], axis=0)
            ES_scenario_mean = np.mean(simulation_data[ES_indices], axis=0) if len(ES_indices) > 0 else np.array([])

            # Find scenarios close to median metric value
            median_metric = np.median(metric)
            median_indices = np.argsort(np.abs(metric - median_metric))[:10]
            median_scenario_mean = np.mean(simulation_data[median_indices], axis=0)

            # Save statistics and indices to dictionary
            statistics_dict[simulation_name] = {
                'stats_df': pd.DataFrame({
                    'Date': simulation_dates,
                    'Median': simulation_median,
                    'Mean': simulation_mean,
                    'Mode': simulation_mode,
                    'StdDev': simulation_std,
                    'Q05': simulation_q05,
                    'Q95': simulation_q95,
                    'MeanAboveQ95': mean_above_q95,
                    'MeanBelowQ05': mean_below_q05,
                    'Mean+StdDev': simulation_mean + simulation_std,
                    'Mean-StdDev': simulation_mean - simulation_std,
                    'Best_Scenario_Mean': best_scenario_mean,
                    'Worst_Scenario_Mean': worst_scenario_mean,
                    'VaR_Scenario_Mean': VaR_scenario_mean,
                    'ES_Scenario_Mean': ES_scenario_mean,
                    'Median_Scenario_Mean': median_scenario_mean
                }),
                'VaR': VaR,
                'ES': ES,
                'simulation_data': simulation_data,
                'best_indices': best_indices,
                'worst_indices': worst_indices,
                'VaR_indices': VaR_indices,
                'ES_indices': ES_indices,
                'median_indices': median_indices
            }

            # Plot graphs
            axs[0].plot(simulation_dates, simulation_q05, label='5th Quantile', color='black', alpha=0.3)
            axs[0].plot(simulation_dates, simulation_q95, label='95th Quantile', color='black', alpha=0.3)
            axs[0].plot(simulation_dates, simulation_mean, label='Mean Across All Simulations', color='red')
            axs[0].fill_between(simulation_dates, simulation_mean - simulation_std, simulation_mean + simulation_std, 
                                color='red', alpha=0.1, label='1 Standard Deviation')

            # Add mean values from both realizations to the plot
            axs[0].plot(simulation_dates, mean_above_q95, label='Mean > 95% Quantile', color='pink', linestyle='--')
            axs[0].plot(simulation_dates, mean_below_q05, label='Mean < 5% Quantile', color='cyan', linestyle='--')

            axs[0].plot(simulation_dates, best_scenario_mean, 
                        label='Mean of Best 10 Scenarios', color='green', linestyle='--')
            axs[0].plot(simulation_dates, worst_scenario_mean, 
                        label='Mean of Worst 10 Scenarios', color='orange', linestyle='--')

            axs[0].plot(simulation_dates, VaR_scenario_mean, 
                        label=f'Mean VaR Scenario ({var_level}%)', color='purple', linestyle='--')
            if ES_scenario_mean.size > 0:
                axs[0].plot(simulation_dates, ES_scenario_mean, 
                            label=f'Mean ES Scenario (>{var_level}%)', color='brown', linestyle='--')

            axs[0].plot(simulation_dates, median_scenario_mean, 
                        label='Mean Scenario (Median)', color='grey', linestyle='--')

            axs[0].set_title(f'{simulation_name.upper()}: Historical Data and Simulations')
            axs[0].set_xlabel('Date')
            axs[0].set_ylabel('Value')
            axs[0].legend()

            # Configure X-axis
            if historical_data_plotted:
                total_dates = pd.concat([data_frame.Period_Date, pd.Series(simulation_dates)])
            else:
                total_dates = pd.Series(simulation_dates)

            xticks_positions = total_dates[::6]
            last_hist_date = data_frame.Period_Date.iloc[-1]
            
            # Add last historical date if not already present
            if last_hist_date not in xticks_positions.values:
                xticks_positions = pd.concat([
                    pd.Series(xticks_positions),
                    pd.Series([last_hist_date])
                ]).sort_values().reset_index(drop=True)
            
            xticks_labels = xticks_positions.dt.strftime('%Y-%m')
            
            axs[0].set_xticks(xticks_positions)
            axs[0].set_xticklabels(xticks_labels, rotation=55)

            axs[0].grid(True)

            if plot_trajectories:
                if historical_data_plotted:
                    axs[1].plot(data_frame.Period_Date, data_frame[historical_column], 
                                label='Historical Data', color='blue')

                # Display random trajectories
                random_indices = np.random.choice(simulation_data.shape[0], min(10, simulation_data.shape[0]), replace=False)
                for idx in random_indices:
                    axs[1].plot(simulation_dates, simulation_data[idx, :], alpha=0.5, color='lightgrey')

                # Display scenarios from both realizations
                for idx in best_indices:
                    axs[1].plot(simulation_dates, simulation_data[idx, :], color='green', alpha=0.3)
                for idx in worst_indices:
                    axs[1].plot(simulation_dates, simulation_data[idx, :], color='orange', alpha=0.3)
                for idx in VaR_indices:
                    axs[1].plot(simulation_dates, simulation_data[idx, :], color='purple', alpha=0.3)
                for idx in ES_indices:
                    axs[1].plot(simulation_dates, simulation_data[idx, :], color='brown', alpha=0.3)
                for idx in median_indices:
                    axs[1].plot(simulation_dates, simulation_data[idx, :], color='grey', alpha=0.3)

                axs[1].set_title(f'{simulation_name.upper()}: Scenario Trajectories')
                axs[1].set_xlabel('Date')
                axs[1].set_ylabel('Value')
                axs[1].set_xticks(xticks_positions)
                axs[1].set_xticklabels(xticks_labels, rotation=90)

                axs[1].grid(True)

            plt.tight_layout()
            pdf.savefig(fig)
            plt.close(fig)

    # Write to Excel using saved data
    with pd.ExcelWriter(output_excel) as writer:
        for simulation_name, data in statistics_dict.items():
            stats_df = data['stats_df']
            simulation_data = data['simulation_data']
            best_indices = data['best_indices']
            worst_indices = data['worst_indices']
            VaR_indices = data['VaR_indices']
            ES_indices = data['ES_indices']
            median_indices = data['median_indices']

            # Save statistics
            stats_df.to_excel(writer, sheet_name=simulation_name, index=False)

            # Save best 10 scenarios
            best_10_data = simulation_data[best_indices]
            best_10_df = pd.DataFrame(best_10_data, columns=simulation_dates[:best_10_data.shape[1]])
            best_10_df.to_excel(writer, sheet_name=f"{simulation_name}_Best_10", index=False)

            # Save worst 10 scenarios
            worst_10_data = simulation_data[worst_indices]
            worst_10_df = pd.DataFrame(worst_10_data, columns=simulation_dates[:worst_10_data.shape[1]])
            worst_10_df.to_excel(writer, sheet_name=f"{simulation_name}_Worst_10", index=False)

            # Save VaR scenarios
            VaR_data = simulation_data[VaR_indices]
            VaR_df = pd.DataFrame(VaR_data, columns=simulation_dates[:VaR_data.shape[1]])
            VaR_df.to_excel(writer, sheet_name=f"{simulation_name}_VaR_Scenarios", index=False)

            # Save ES scenarios
            if len(ES_indices) > 0:
                ES_data = simulation_data[ES_indices]
                ES_df = pd.DataFrame(ES_data, columns=simulation_dates[:ES_data.shape[1]])
                ES_df.to_excel(writer, sheet_name=f"{simulation_name}_ES_Scenarios", index=False)

            # Save median scenarios
            median_data = simulation_data[median_indices]
            median_df = pd.DataFrame(median_data, columns=simulation_dates[:median_data.shape[1]])
            median_df.to_excel(writer, sheet_name=f"{simulation_name}_Median_Scenarios", index=False)

        # Save statistics description on separate sheet
        description_df = pd.DataFrame(list(stats_description.items()), columns=['Statistic', 'Description'])
        description_df.to_excel(writer, sheet_name='Statistics Description', index=False)

    print(f"Charts saved to {output_pdf}")
    print(f"Statistics saved to {output_excel}")

def load_params_from_excel(excel_file_path):
    loaded_sim_params = {}
    excel_file = pd.ExcelFile(excel_file_path)
    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

        # If it's a single value
        if df.shape[0] == 1 and df.shape[1] == 1:
            value = df.iloc[0, 0]
        elif df.shape[0] == 1:
            # Convert one row to dictionary
            print("sheet_name", sheet_name)
            value = df.iloc[0].to_dict()
            
        else:
            value = df

        if isinstance(value, str):
            try:
                value = eval(value)
            except:
                pass

        loaded_sim_params[sheet_name] = value

    return loaded_sim_params

def generate_macro3(irshock=0, free=True):
    excel_file_path = './data_and_params.xlsx'

    full_path = os.path.abspath(excel_file_path)
    # print(f"Reading file from path: {full_path}"

    loaded_sim_params = load_params_from_excel(full_path)

    # Convert some data back to required format
    # 'macro_data'
    if 'Period_Date' in loaded_sim_params:
        df = loaded_sim_params['macro_data']
        if 'Period_Date' in df.columns:
            df['Period_Date'] = pd.to_datetime(df['Period_Date'])
        loaded_sim_params['macro_data'] = df
    # 'Date'
    if 'History' in loaded_sim_params:
        df1 = loaded_sim_params['History']
        if 'Date' in df1.columns:
            df1['Date'] = pd.to_datetime(df1['Date'])
        loaded_sim_params['History'] = df1       

    # dates
    date_keys = ['macro_date', 'evaluation_start', 'EvaluationDate'] #, 'start_date' kbd_date
    for key in date_keys:
        if key in loaded_sim_params:
            value = loaded_sim_params[key]
            if isinstance(value, pd.DataFrame):
                # Assume value is stored in first cell
                loaded_sim_params[key] = pd.to_datetime(value.iloc[1, 1])
            else:
                # loaded_sim_params[key] = pd.to_datetime(value[key])
                # key = value[key]
                loaded_sim_params[key] = pd.to_datetime(value)
    
    simulator = simul_generate.Simulator(loaded_sim_params, free=free)
    export, res_vec = simulator.generate_macro3(irshock, free)

    return export, res_vec, loaded_sim_params

export, res_vec, loaded_sim_params = generate_macro3()
plot_simulation_with_historical_and_trajectories(loaded_sim_params['macro_data'], res_vec, loaded_sim_params['portfolio_params']['kbd_date'],  plot_trajectories= False, output_pdf=f'macro_factors{ts}.pdf')

sheet_name Coefficients
sheet_name Coefficients030325
sheet_name Coefficients130125
sheet_name cc
sheet_name volatility
sheet_name portfolio_params
plan_max  59
rnd: RandomState(MT19937)


'N_scr'

10000

'NumberOfMonteCarloScenarios'

10000

Coefficients: {'B0': 1409.96, 'B1': 510.44, 'B2': -334.49, 'TAU': 0.57, 'G1': -8.11, 'G2': -5.29, 'G3': 7.88, 'G4': 5.09, 'G5': -1.1, 'G6': 6.13, 'G7': -2.63, 'G8': 0.0, 'G9': 0.0}
## Generate Macro ##
seed: 10
macroseedlist: [5]
## CCIR and montecarlo ##
Seed: 10
len(macro_data['cpi_yoy'][-15:]): 120
len(ind3MR) : 217
(int(12 * T) - 15)/12 = 17.5


'macro_cir'

array([[0.21080038, 0.21080038, 0.22482225, ..., 0.10715113, 0.10723398,
        0.12144744],
       [0.21080038, 0.21080038, 0.21430298, ..., 0.09156063, 0.08163471,
        0.09206339],
       [0.21080038, 0.21080038, 0.17648482, ..., 0.07971429, 0.07760626,
        0.08203771],
       ...,
       [0.21080038, 0.21080038, 0.21314957, ..., 0.07851863, 0.07833758,
        0.06271067],
       [0.21080038, 0.21080038, 0.22343715, ..., 0.05364719, 0.04802581,
        0.0535978 ],
       [0.21080038, 0.21080038, 0.2219348 , ..., 0.04684225, 0.05099646,
        0.05516683]])

Charts saved to macro_factors20250630_175833.pdf
Statistics saved to simulation_statistics.xlsx


In [4]:
def plot_simulation_with_historical_and_trajectories(
    data_frame, simulation_dict, start_date, plot_trajectories=True, 
    output_pdf='macro_factors.pdf', 
    output_excel='simulation_statistics.xlsx',
    trim_simulation=True):

    mapping = {
        'cost': 'cost_qoq',
        'cir': 'rate',
        'hpi': 'hpi_pr_qoq',
        'mortgage_rate': 'MR',
        'hpi_second': 'hpi_yoy'
    }
    
    if trim_simulation:
        trimmed_simulation_dict = {key: data[:, :-15] for key, data in simulation_dict.items()}
    else:
        trimmed_simulation_dict = simulation_dict

    any_key = next(iter(trimmed_simulation_dict))
    end_simulation_months = trimmed_simulation_dict[any_key].shape[1]
    simulation_dates = pd.date_range(start=start_date, periods=end_simulation_months, freq='M')
    simulation_dates = pd.to_datetime(simulation_dates)
    statistics_dict = {}
    stats_description = {
        'Median': 'Медиана значений симуляций в каждый момент времени',
        'Mean': 'Среднее значение симуляций в каждый момент времени',
        'Mode': 'Мода значений симуляций в каждый момент времени',
        'StdDev': 'Стандартное отклонение значений симуляций в каждый момент времени',
        'Q05': '5-й процентиль значений симуляций в каждый момент времени',
        'Q95': '95-й процентиль значений симуляций в каждый момент времени',
        'MeanAboveQ95': 'Среднее значение симуляций выше 95-го процентиля в каждый момент времени',
        'MeanBelowQ05': 'Среднее значение симуляций ниже 5-го процентиля в каждый момент времени',
        'Mean+StdDev': 'Сумма среднего значения и стандартного отклонения в каждый момент времени',
        'Mean-StdDev': 'Разница среднего значения и стандартного отклонения в каждый момент времени',
        'Best_Scenario_Mean': 'Среднее значение лучших сценариев',
        'Worst_Scenario_Mean': 'Среднее значение худших сценариев',
        'VaR_Scenario_Mean': 'Среднее значение сценариев VaR на уровне 95%',
        'ES_Scenario_Mean': 'Среднее значение сценариев ES выше VaR 95%',
        'Median_Scenario_Mean': 'Среднее значение сценариев, близких к медиане'
    }

    with PdfPages(output_pdf) as pdf:
        for simulation_name, simulation_data in trimmed_simulation_dict.items():
            fig, axs = plt.subplots(2 if plot_trajectories else 1, 1, 
                                    figsize=(18, 12 if plot_trajectories else 6), sharex=True)
            if not isinstance(axs, np.ndarray):
                axs = [axs]

            historical_data_plotted = False

            if simulation_name in data_frame.columns:
                historical_column = simulation_name
            elif simulation_name in mapping and mapping[simulation_name] in data_frame.columns:
                historical_column = mapping[simulation_name]
            else:
                historical_column = None 

            if historical_column is not None:
                axs[0].plot(data_frame.Period_Date, data_frame[historical_column], 
                            label='Исторические данные', color='blue')
                historical_data_plotted = True

            simulation_mean = np.mean(simulation_data, axis=0)
            simulation_median = np.median(simulation_data, axis=0)
            simulation_mode = mode(simulation_data, axis=0)[0].squeeze()
            simulation_q05 = np.percentile(simulation_data, 5, axis=0)
            simulation_q95 = np.percentile(simulation_data, 95, axis=0)
            simulation_std = np.std(simulation_data, axis=0)

            metric = np.mean(simulation_data[:, :25], axis=1)

            metric_q95 = np.percentile(metric, 95)
            metric_q05 = np.percentile(metric, 5)
            
            mean_above_q95 = np.mean(simulation_data[metric > metric_q95], axis=0)
            mean_below_q05 = np.mean(simulation_data[metric < metric_q05], axis=0)
            positive_metric = simulation_name in ['gdp_yoy']
            if positive_metric:
                best_indices = np.argsort(-metric)[:10]  # Индексы 10 наибольших значений
                worst_indices = np.argsort(metric)[:10]  # Индексы 10 наименьших значений
            else:
                best_indices = np.argsort(metric)[:10]   # Индексы 10 наименьших значений
                worst_indices = np.argsort(-metric)[:10] # Индексы 10 наибольших значений

            best_scenario_mean = np.mean(simulation_data[best_indices], axis=0)
            worst_scenario_mean = np.mean(simulation_data[worst_indices], axis=0)

            # Расчет VaR и ES из второй реализации
            var_level = 95
            VaR = np.percentile(metric, var_level)
            ES_indices = np.where(metric > VaR)[0]
            VaR_indices = np.argsort(np.abs(metric - VaR))[:5]  # 5 сценариев, ближайших к VaR
            ES = metric[metric > VaR].mean() if len(ES_indices) > 0 else np.nan

            VaR_scenario_mean = np.mean(simulation_data[VaR_indices], axis=0)
            ES_scenario_mean = np.mean(simulation_data[ES_indices], axis=0) if len(ES_indices) > 0 else np.array([])

            # Находим сценарии, близкие к медианному значению метрики
            median_metric = np.median(metric)
            median_indices = np.argsort(np.abs(metric - median_metric))[:10]
            median_scenario_mean = np.mean(simulation_data[median_indices], axis=0)

            # Сохранение статистик и индексов в словарь
            statistics_dict[simulation_name] = {
                'stats_df': pd.DataFrame({
                    'Date': simulation_dates,
                    'Median': simulation_median,
                    'Mean': simulation_mean,
                    'Mode': simulation_mode,
                    'StdDev': simulation_std,
                    'Q05': simulation_q05,
                    'Q95': simulation_q95,
                    'MeanAboveQ95': mean_above_q95,
                    'MeanBelowQ05': mean_below_q05,
                    'Mean+StdDev': simulation_mean + simulation_std,
                    'Mean-StdDev': simulation_mean - simulation_std,
                    'Best_Scenario_Mean': best_scenario_mean,
                    'Worst_Scenario_Mean': worst_scenario_mean,
                    'VaR_Scenario_Mean': VaR_scenario_mean,
                    'ES_Scenario_Mean': ES_scenario_mean,
                    'Median_Scenario_Mean': median_scenario_mean
                }),
                'VaR': VaR,
                'ES': ES,
                'simulation_data': simulation_data,
                'best_indices': best_indices,
                'worst_indices': worst_indices,
                'VaR_indices': VaR_indices,
                'ES_indices': ES_indices,
                'median_indices': median_indices
            }

            # Построение 
            axs[0].plot(simulation_dates, simulation_q05, label='5-й квантиль', color='black', alpha=0.3)
            axs[0].plot(simulation_dates, simulation_q95, label='95-й квантиль', color='black', alpha=0.3)
            axs[0].plot(simulation_dates, simulation_mean, label='Среднее по всем симуляциям', color='red')
            axs[0].fill_between(simulation_dates, simulation_mean - simulation_std, simulation_mean + simulation_std, 
                                color='red', alpha=0.1, label='1 стандартное отклонение')
            axs[0].plot(simulation_dates, mean_above_q95, label='Среднее > 95% кв.', color='pink', linestyle='--')
            axs[0].plot(simulation_dates, mean_below_q05, label='Среднее < 5% кв.', color='cyan', linestyle='--')

            axs[0].plot(simulation_dates, best_scenario_mean, 
                        label='Среднее лучших 10 сценариев', color='green', linestyle='--')
            axs[0].plot(simulation_dates, worst_scenario_mean, 
                        label='Среднее худших 10 сценариев', color='orange', linestyle='--')

            axs[0].plot(simulation_dates, VaR_scenario_mean, 
                        label=f'Средний сценарий VaR ({var_level}%)', color='purple', linestyle='--')
            if ES_scenario_mean.size > 0:
                axs[0].plot(simulation_dates, ES_scenario_mean, 
                            label=f'Средний сценарий ES (>{var_level}%)', color='brown', linestyle='--')

            axs[0].plot(simulation_dates, median_scenario_mean, 
                        label='Средний сценарий (медианный)', color='grey', linestyle='--')

            axs[0].set_title(f'{simulation_name.upper()}: Исторические данные и симуляции')
            axs[0].set_xlabel('Дата')
            axs[0].set_ylabel('Значение')
            axs[0].legend()

            # Настройка осей X
            if historical_data_plotted:
                total_dates = pd.concat([data_frame.Period_Date, pd.Series(simulation_dates)])
            else:
                total_dates = pd.Series(simulation_dates)

            xticks_positions = total_dates[::6]
            xticks_labels = xticks_positions.dt.strftime('%Y-%m')
            axs[0].set_xticks(xticks_positions)
            axs[0].set_xticklabels(xticks_labels, rotation=55)

            axs[0].grid(True)

            if plot_trajectories:
                if historical_data_plotted:
                    axs[1].plot(data_frame.Period_Date, data_frame[historical_column], 
                                label='Исторические данные', color='blue')

                # Отображаем случайные траектории
                random_indices = np.random.choice(simulation_data.shape[0], min(10, simulation_data.shape[0]), replace=False)
                for idx in random_indices:
                    axs[1].plot(simulation_dates, simulation_data[idx, :], alpha=0.5, color='lightgrey')

                # Отображаем сценарии из обеих реализаций
                for idx in best_indices:
                    axs[1].plot(simulation_dates, simulation_data[idx, :], color='green', alpha=0.3)
                for idx in worst_indices:
                    axs[1].plot(simulation_dates, simulation_data[idx, :], color='orange', alpha=0.3)
                for idx in VaR_indices:
                    axs[1].plot(simulation_dates, simulation_data[idx, :], color='purple', alpha=0.3)
                for idx in ES_indices:
                    axs[1].plot(simulation_dates, simulation_data[idx, :], color='brown', alpha=0.3)
                for idx in median_indices:
                    axs[1].plot(simulation_dates, simulation_data[idx, :], color='grey', alpha=0.3)

                axs[1].set_title(f'{simulation_name.upper()}: Траектории сценариев')
                axs[1].set_xlabel('Дата')
                axs[1].set_ylabel('Значение')
                axs[1].set_xticks(xticks_positions)
                axs[1].set_xticklabels(xticks_labels, rotation=90)

                axs[1].grid(True)

            plt.tight_layout()
            pdf.savefig(fig)
            plt.close(fig)

    # Запись в Excel с использованием сохраненных данных
    with pd.ExcelWriter(output_excel) as writer:
        for simulation_name, data in statistics_dict.items():
            stats_df = data['stats_df']
            simulation_data = data['simulation_data']
            best_indices = data['best_indices']
            worst_indices = data['worst_indices']
            VaR_indices = data['VaR_indices']
            ES_indices = data['ES_indices']
            median_indices = data['median_indices']

            # Сохраняем статистики
            stats_df.to_excel(writer, sheet_name=simulation_name, index=False)

            # Сохраняем лучшие 10 сценариев
            best_10_data = simulation_data[best_indices]
            best_10_df = pd.DataFrame(best_10_data, columns=simulation_dates[:best_10_data.shape[1]])
            best_10_df.to_excel(writer, sheet_name=f"{simulation_name}_Best_10", index=False)

            # Сохраняем худшие 10 сценариев
            worst_10_data = simulation_data[worst_indices]
            worst_10_df = pd.DataFrame(worst_10_data, columns=simulation_dates[:worst_10_data.shape[1]])
            worst_10_df.to_excel(writer, sheet_name=f"{simulation_name}_Worst_10", index=False)

            # Сохраняем сценарии VaR
            VaR_data = simulation_data[VaR_indices]
            VaR_df = pd.DataFrame(VaR_data, columns=simulation_dates[:VaR_data.shape[1]])
            VaR_df.to_excel(writer, sheet_name=f"{simulation_name}_VaR_Scenarios", index=False)

            # Сохраняем сценарии ES
            if len(ES_indices) > 0:
                ES_data = simulation_data[ES_indices]
                ES_df = pd.DataFrame(ES_data, columns=simulation_dates[:ES_data.shape[1]])
                ES_df.to_excel(writer, sheet_name=f"{simulation_name}_ES_Scenarios", index=False)

            # Сохраняем медианные сценарии
            median_data = simulation_data[median_indices]
            median_df = pd.DataFrame(median_data, columns=simulation_dates[:median_data.shape[1]])
            median_df.to_excel(writer, sheet_name=f"{simulation_name}_Median_Scenarios", index=False)

        # Сохраняем описание статистик на отдельный лист
        description_df = pd.DataFrame(list(stats_description.items()), columns=['Статистика', 'Описание'])
        description_df.to_excel(writer, sheet_name='Описание статистик', index=False)

    print(f"Графики сохранены в {output_pdf}")
    print(f"Статистика сохранена в {output_excel}")

In [5]:
def load_params_from_excel(excel_file_path):
    loaded_sim_params = {}
    excel_file = pd.ExcelFile(excel_file_path)
    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

        # если это одиночное значение
        if df.shape[0] == 1 and df.shape[1] == 1:
            value = df.iloc[0, 0]
        elif df.shape[0] == 1:
            # преобразуем одну строку в словарь
            print("sheet_name", sheet_name)
            value = df.iloc[0].to_dict()
            
        else:
            value = df

        if isinstance(value, str):
            try:
                value = eval(value)
            except:
                pass

        loaded_sim_params[sheet_name] = value

    return loaded_sim_params

def generate_macro3(irshock=0, free=True):
    excel_file_path = './sim_params.xlsx'
    full_path = os.path.abspath(excel_file_path)
    print(f"Читаю файл по пути: {full_path}")


    loaded_sim_params = load_params_from_excel(full_path)

    if 'Period_Date' in loaded_sim_params:
        df = loaded_sim_params['macro_data']
        if 'Period_Date' in df.columns:
            df['Period_Date'] = pd.to_datetime(df['Period_Date'])
        loaded_sim_params['macro_data'] = df
    # 'Date'
    if 'History' in loaded_sim_params:
        df1 = loaded_sim_params['History']
        if 'Date' in df1.columns:
            df1['Date'] = pd.to_datetime(df1['Date'])
        loaded_sim_params['History'] = df1       

    #  даты
    date_keys = ['macro_date', 'evaluation_start', 'EvaluationDate'] #, 'start_date' kbd_date
    for key in date_keys:
        if key in loaded_sim_params:
            value = loaded_sim_params[key]
            if isinstance(value, pd.DataFrame):
                loaded_sim_params[key] = pd.to_datetime(value.iloc[1, 1])
            else:

                loaded_sim_params[key] = pd.to_datetime(value)
    
    simulator = simul_generate.Simulator(loaded_sim_params, free=free)
    export, res_vec = simulator.generate_macro3(irshock, free)
    return export, res_vec, loaded_sim_params

export, res_vec, loaded_sim_params = generate_macro3()
plot_simulation_with_historical_and_trajectories(loaded_sim_params['macro_data'], res_vec, loaded_sim_params['portfolio_params']['kbd_date'],  plot_trajectories= False, output_pdf='macro_factors.pdf')

Читаю файл по пути: /Users/andrei/Desktop/статья/статья2/статья 3/алгоритмы и код/code/macro_model_2024-calculator_vsego/kalkulator_vsego/simul/sim_params.xlsx


FileNotFoundError: [Errno 2] No such file or directory: '/Users/andrei/Desktop/статья/статья2/статья 3/алгоритмы и код/code/macro_model_2024-calculator_vsego/kalkulator_vsego/simul/sim_params.xlsx'